In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RectBivariateSpline
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D

# Black-Scholes 看涨期权定价公式
def black_scholes_call(S, K, T, r, sigma, q=0):
    if T <= 0:
        return max(S - K, 0)
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

# 计算局部波动率函数
def compute_local_volatility(S, r, q, K_grid, T_grid, iv_surface):
    """
    从隐含波动率曲面计算局部波动率
    """
    # 创建期权价格曲面
    call_prices = np.zeros_like(iv_surface)
    for i, T in enumerate(T_grid):
        for j, K in enumerate(K_grid):
            call_prices[i, j] = black_scholes_call(S, K, T, r, iv_surface[i, j], q)
    
    # 计算偏导数
    dC_dT = np.zeros_like(iv_surface)
    dC_dK = np.zeros_like(iv_surface)
    d2C_dK2 = np.zeros_like(iv_surface)
    
    dT = T_grid[1] - T_grid[0] if len(T_grid) > 1 else 0.1
    dK = K_grid[1] - K_grid[0] if len(K_grid) > 1 else 1.0
    
    for i in range(len(T_grid)):
        for j in range(len(K_grid)):
            # 对T的偏导数 (∂C/∂T)
            if i == 0:
                dC_dT[i, j] = (call_prices[i+1, j] - call_prices[i, j]) / dT
            elif i == len(T_grid)-1:
                dC_dT[i, j] = (call_prices[i, j] - call_prices[i-1, j]) / dT
            else:
                dC_dT[i, j] = (call_prices[i+1, j] - call_prices[i-1, j]) / (2*dT)
            
            # 对K的一阶偏导数 (∂C/∂K)
            if j == 0:
                dC_dK[i, j] = (call_prices[i, j+1] - call_prices[i, j]) / dK
            elif j == len(K_grid)-1:
                dC_dK[i, j] = (call_prices[i, j] - call_prices[i, j-1]) / dK
            else:
                dC_dK[i, j] = (call_prices[i, j+1] - call_prices[i, j-1]) / (2*dK)
            
            # 对K的二阶偏导数 (∂²C/∂K²)
            if j == 0:
                d2C_dK2[i, j] = (call_prices[i, j+2] - 2*call_prices[i, j+1] + call_prices[i, j]) / (dK**2)
            elif j == len(K_grid)-1:
                d2C_dK2[i, j] = (call_prices[i, j] - 2*call_prices[i, j-1] + call_prices[i, j-2]) / (dK**2)
            else:
                d2C_dK2[i, j] = (call_prices[i, j+1] - 2*call_prices[i, j] + call_prices[i, j-1]) / (dK**2)
    
    # 应用Dupire公式
    local_vol_surface = np.zeros_like(iv_surface)
    for i in range(len(T_grid)):
        for j in range(len(K_grid)):
            numerator = 2 * (dC_dT[i, j] + r*K_grid[j]*dC_dK[i, j] + q*call_prices[i, j])
            denominator = (K_grid[j]**2) * d2C_dK2[i, j]
            
            # 避免除零错误和负值
            if denominator > 1e-8 and numerator > 0:  # 添加阈值以避免数值问题
                local_vol_squared = numerator / denominator
                if local_vol_squared > 0:
                    local_vol_surface[i, j] = np.sqrt(local_vol_squared)
                else:
                    local_vol_surface[i, j] = np.nan
            else:
                local_vol_surface[i, j] = np.nan
    
    return local_vol_surface

# 创建示例数据
S = 100.0  # 当前标的资产价格
r = 0.05   # 无风险利率
q = 0.02   # 股息率

# 创建网格
K_grid = np.linspace(80, 120, 41)  # 行权价从80到120
T_grid = np.linspace(0.1, 2.0, 20)  # 到期时间从0.1年到2年

# 创建隐含波动率曲面 - 修正为正确的微笑形状
K_mesh, T_mesh = np.meshgrid(K_grid, T_grid)

# 创建一个典型的波动率微笑：中间低，两边高
moneyness = (K_mesh - S) / S  # 计算价外程度
iv_surface = 0.2 + 0.1 * moneyness**2  # 抛物线形状的微笑

# 添加期限结构 - 短期波动率微笑更明显
time_factor = 0.8 + 0.2 * np.exp(-T_mesh)  # 期限结构因子
iv_surface = iv_surface * time_factor

# 计算局部波动率
local_vol_surface = compute_local_volatility(S, r, q, K_grid, T_grid, iv_surface)

# 创建3D图形 - 调整视角使短期期限正对观察者
fig = plt.figure(figsize=(16, 8))

# 绘制隐含波动率曲面
ax1 = fig.add_subplot(121, projection='3d')
surf1 = ax1.plot_surface(K_mesh, T_mesh, iv_surface, cmap='viridis', edgecolor='none', alpha=0.8)
ax1.set_title('Implied Volatility Surface (Smile Shape)\nShort Term Maturity Facing Viewer')
ax1.set_xlabel('Strike Price (K)')
ax1.set_ylabel('Time to Maturity (T)')
ax1.set_zlabel('Implied Volatility')
# 调整视角 - 使短期期限正对观察者
ax1.view_init(elev=20, azim=-70)  # 调整仰角和方位角
fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=10)

# 绘制局部波动率曲面
ax2 = fig.add_subplot(122, projection='3d')
surf2 = ax2.plot_surface(K_mesh, T_mesh, local_vol_surface, cmap='plasma', edgecolor='none', alpha=0.8)
ax2.set_title('Local Volatility Surface\nShort Term Maturity Facing Viewer')
ax2.set_xlabel('Strike Price (K)')
ax2.set_ylabel('Time to Maturity (T)')
ax2.set_zlabel('Local Volatility')
# 调整视角 - 使短期期限正对观察者
ax2.view_init(elev=20, azim=-70)  # 调整仰角和方位角
fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=10)

plt.tight_layout()
plt.show()

# 绘制二维切片图 - 特别突出短期期限
plt.figure(figsize=(12, 6))
selected_T_indices = [0, len(T_grid)//3, 2*len(T_grid)//3, len(T_grid)-1]  # 选择四个不同的到期时间
colors = ['b', 'r', 'g', 'm']
labels = ['Very Short-term (T=0.1y)', 'Short-term', 'Medium-term', 'Long-term (T=2.0y)']

for i, idx in enumerate(selected_T_indices):
    T = T_grid[idx]
    linewidth = 3 if i == 0 else 2  # 特别加粗最短期限的线
    plt.plot(K_grid, iv_surface[idx, :], f'{colors[i]}--', 
             label=f'IV ({labels[i]})', linewidth=linewidth)
    plt.plot(K_grid, local_vol_surface[idx, :], f'{colors[i]}-', 
             label=f'LV ({labels[i]})', linewidth=linewidth)

plt.axvline(x=S, color='k', linestyle=':', alpha=0.5, label='ATM (S=100)')
plt.xlabel('Strike Price')
plt.ylabel('Volatility')
plt.title('Implied vs Local Volatility: Smile Shape (Emphasizing Short Term)')
plt.legend()
plt.grid(True)
plt.show()

# 特别关注短期期限的3D图
fig = plt.figure(figsize=(14, 6))

# 隐含波动率曲面 - 只显示短期部分
ax1 = fig.add_subplot(121, projection='3d')
# 只选择前1/3的期限（短期）
short_term_mask = T_mesh <= 0.8
K_short = np.where(short_term_mask, K_mesh, np.nan)
T_short = np.where(short_term_mask, T_mesh, np.nan)
IV_short = np.where(short_term_mask, iv_surface, np.nan)

surf1 = ax1.plot_surface(K_short, T_short, IV_short, cmap='viridis', edgecolor='none', alpha=0.8)
ax1.set_title('Implied Volatility Surface (Short Term Only)')
ax1.set_xlabel('Strike Price (K)')
ax1.set_ylabel('Time to Maturity (T)')
ax1.set_zlabel('Implied Volatility')
ax1.view_init(elev=25, azim=-60)
fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=10)

# 局部波动率曲面 - 只显示短期部分
ax2 = fig.add_subplot(122, projection='3d')
LV_short = np.where(short_term_mask, local_vol_surface, np.nan)
surf2 = ax2.plot_surface(K_short, T_short, LV_short, cmap='plasma', edgecolor='none', alpha=0.8)
ax2.set_title('Local Volatility Surface (Short Term Only)')
ax2.set_xlabel('Strike Price (K)')
ax2.set_ylabel('Time to Maturity (T)')
ax2.set_zlabel('Local Volatility')
ax2.view_init(elev=25, azim=-60)
fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=10)

plt.tight_layout()
plt.show()

# 打印一些统计信息
print(f"Implied Volatility Range: {np.nanmin(iv_surface):.3f} - {np.nanmax(iv_surface):.3f}")
print(f"Local Volatility Range: {np.nanmin(local_vol_surface):.3f} - {np.nanmax(local_vol_surface):.3f}")
print(f"Short-term (T=0.1y) ATM Implied Volatility: {iv_surface[0, np.argmin(np.abs(K_grid - S))]:.3f}")
print(f"Short-term (T=0.1y) ATM Local Volatility: {local_vol_surface[0, np.argmin(np.abs(K_grid - S))]:.3f}")